## Importing Dependencies

In [2]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [4]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import Counter
import nltk
from nltk.corpus import stopwords

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.stem import *

In [7]:
stemmer = PorterStemmer()

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.metrics import classification_report

In [10]:
from sklearn.naive_bayes import MultinomialNB

In [11]:
pip install lightgbm


Note: you may need to restart the kernel to use updated packages.


In [12]:
import lightgbm as lgb

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content= stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content

## Loading Data

In [14]:
import os
print(os.getcwd())  # Prints the current working directory


c:\Users\HP\Desktop\sentiment


In [15]:
import pandas as pd
file_path = 'twitter_data.csv'
df = pd.read_csv(file_path, encoding="ISO-8859-1")



In [16]:
import pandas as pd

column_names = ["target", "id", "date", "flag","user","text"]

df = pd.read_csv("twitter_data.csv", names=column_names, header=None, encoding="ISO-8859-1")



In [17]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df["stemmed_text"] = df["text"].apply(lambda x: stemming(x))

In [ ]:
print(df.columns)


Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')


In [ ]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
df.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
X = df['stemmed_content'].values

KeyError: 'stemmed_content'

In [ ]:
Y = df['target'].values

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.2 , stratify=Y , random_state=2)

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Model Training

In [ ]:
model1 = LogisticRegression(C=1,class_weight='balanced',max_iter=200,tol=0.0001,solver='saga')

In [ ]:
model1.fit(X_train, Y_train)

LogisticRegression(C=1, class_weight='balanced', max_iter=200, solver='saga')

In [ ]:
model2 = MultinomialNB()

In [ ]:
model2.fit(X_train, Y_train)

MultinomialNB()

In [ ]:
train_data = lgb.Dataset(X_train, label=Y_train)
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_error',  # Binary error metric
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'num_leaves': 31,  # Number of leaves in one tree
    'learning_rate': 0.05,  # Learning rate
    'feature_fraction': 0.9  # Fraction of features used for each tree
}

# Train the model
model3 = lgb.train(params, train_data, num_boost_round=100)

[LightGBM] [Info] Number of positive: 639813, number of negative: 639791
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 35.505602 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 316601
[LightGBM] [Info] Number of data points in the train set: 1279604, number of used features: 22202
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500009 -> initscore=0.000034
[LightGBM] [Info] Start training from score 0.000034


## Model Evaluation

### Logistic Regression

In [ ]:
X_train_prediction1 = model1.predict(X_train)
training_data_accuracy1 = accuracy_score(Y_train,X_train_prediction1)

In [ ]:
print("Accuracy Score on Training Data: ",training_data_accuracy1*100,"%")

Accuracy Score on Training Data:  81.01334475353312 %


In [ ]:
X_test_prediction1 = model1.predict(X_test)
test_data_accuracy1 = accuracy_score(Y_test,X_test_prediction1)

In [ ]:
print("Accuracy Score on Test Data ",test_data_accuracy1*100,"%")

Accuracy Score on Test Data  77.87596787756212 %


### MultinomialNB

In [ ]:
X_train_prediction2 = model2.predict(X_train)
training_data_accuracy2 = accuracy_score(Y_train,X_train_prediction2)

In [ ]:
print("Accuracy Score on Training Data: ",training_data_accuracy2*100,"%")

Accuracy Score on Training Data:  81.99130355953874 %


In [ ]:
X_test_prediction2 = model2.predict(X_test)
test_data_accuracy2 = accuracy_score(Y_test,X_test_prediction2)

In [ ]:
print("Accuracy Score on Test Data ",test_data_accuracy2*100,"%")

Accuracy Score on Test Data  75.60557797568623 %


### Gradient Boosting

In [ ]:
X_train_prediction3 = model3.predict(X_train)
X_train_prediction3_binary = (X_train_prediction3 >= 0.5).astype(int) 
training_data_accuracy3 = accuracy_score(Y_train,X_train_prediction3_binary)

In [ ]:
print("Accuracy Score on Training Data: ",training_data_accuracy3*100,"%")

Accuracy Score on Training Data:  72.44553783826872 %


In [ ]:
X_test_prediction3 = model3.predict(X_test)
X_test_prediction3_binary = (X_test_prediction3 >= 0.5).astype(int) 
test_data_accuracy3 = accuracy_score(Y_test,X_test_prediction3_binary)

In [ ]:
print("Accuracy Score on Test Data ",test_data_accuracy3*100,"%")

Accuracy Score on Test Data  72.36395009706128 %


## Result Table

In [ ]:
models = ['Logistic Regression', 'MultinomialNB', 'Gradient Boosting']

train_accuracy = [round(training_data_accuracy1*100,2), round(training_data_accuracy2*100,2), round(training_data_accuracy3*100,2)]

test_accuracy = [round(test_data_accuracy1*100,2), round(test_data_accuracy2*100,2), round(test_data_accuracy3*100,2)]  

results = pd.DataFrame({
    'Model': models,
    'Training Accuracy': train_accuracy,
    'Test Accuracy': test_accuracy
})

print(results)

                 Model  Training Accuracy  Test Accuracy
0  Logistic Regression              81.01          77.88
1        MultinomialNB              81.99          75.61
2    Gradient Boosting              72.45          72.36


## Classification Reports

### Logistic Regression

In [ ]:
report1 = classification_report(Y_test,X_test_prediction1)
print(report1)

              precision    recall  f1-score   support

           0       0.79      0.76      0.77    159948
           1       0.77      0.80      0.78    159953

    accuracy                           0.78    319901
   macro avg       0.78      0.78      0.78    319901
weighted avg       0.78      0.78      0.78    319901



### MultinomialNB

In [ ]:
report2 = classification_report(Y_test,X_test_prediction2)
print(report2)

              precision    recall  f1-score   support

           0       0.74      0.78      0.76    159948
           1       0.77      0.73      0.75    159953

    accuracy                           0.76    319901
   macro avg       0.76      0.76      0.76    319901
weighted avg       0.76      0.76      0.76    319901



### Gradient Boosting

In [ ]:
report3 = classification_report(Y_test,X_test_prediction3_binary)
print(report3)

              precision    recall  f1-score   support

           0       0.78      0.62      0.69    159948
           1       0.69      0.83      0.75    159953

    accuracy                           0.72    319901
   macro avg       0.73      0.72      0.72    319901
weighted avg       0.73      0.72      0.72    319901

